In [121]:
import numpy as np
import pandas as pd
from molvs import standardize_smiles
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler

In [122]:
df = pd.read_csv('data/ionized.tsv', delimiter='\t')
df['smiles'] = df['smiles'].apply(standardize_smiles)

[15:32:14] WARNING: not removing hydrogen atom without neighbors
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7
[15:32:49] Can't kekulize mol.  Unkekulized atoms: 3 7


In [123]:
#Remove duplicated smiles and unite kegg ids in list for each molecule
df = df.groupby("smiles")["kegg_id"].apply(list).reset_index()
df

,smiles,kegg_id
0,BrC1=CC2OC2C=C1,[C14839]
1,BrC1=CC=CC2OC12,[C14840]
2,BrCCBr,[C11088]
3,Brc1ccc(Oc2ccc(Br)cc2Br)c(Br)c1,[C18205]
4,Brc1ccccc1,[C11036]
...,...,...
89478,c1cncc(C2CC[NH2+]C(c3cccnc3)C2)c1,[C12454]
89479,c1nc(CC2CCNCC2)c[n-]1,[C17932]
89480,c1nc(CC2CCNCC2)c[nH]1,[C17932]
89481,c1nc(CC2CC[NH2+]CC2)c[n-]1,[C17932]


In [125]:
df['formal_charge'] = df['smiles'].apply(lambda x: Chem.GetFormalCharge(Chem.MolFromSmiles(x)))
df = df[(df['formal_charge']>-3)&(df['formal_charge']<3)&(df['formal_charge']!=0)]

[15:34:40] WARNING: not removing hydrogen atom without neighbors
/tmp/ipykernel_329794/713347994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['formal_charge'] = df['smiles'].apply(lambda x: Chem.GetFormalCharge(Chem.MolFromSmiles(x)))


In [127]:
search_patterns = [
    '[CX3H1](=O)[#6]', #aldehyde
    '[OX2H][#6X3]=[#6]', #enole
    '[OX2,OX1-][OX2,OX1-]', #peroxyde
    '[O-][C,c;!$([C]=[!#6;!H])]' #not acidic hydroxyl ionized (phenoles and alcohols)
]

def filter_patterns(mol):
    for pattern in search_patterns:
        pattern = Chem.MolFromSmarts(pattern)
        if mol.HasSubstructMatch(pattern):
            return True
    return False

In [128]:
#Filter out banned patterns
df = df[~df['smiles'].apply(lambda x: filter_patterns(Chem.MolFromSmiles(x)))]
df

[15:35:19] WARNING: not removing hydrogen atom without neighbors


,smiles,kegg_id,formal_charge
8,C#CC(=O)[O-],[C00804],-1
10,C#CCC(=O)[O-],[C06144],-1
15,C#CC[NH2+][C@H](C)Cc1ccccc1,[C15476],1
18,C/C(=C/C(=O)O)C(=O)[O-],[C02226],-1
19,C/C(=C/C(=O)[O-])C(=O)O,[C02226],-1
...,...,...,...
89475,c1cncc(C2CCNC(c3ccc[nH+]c3)C2)c1,[C12454],1
89477,c1cncc(C2CC[NH2+]C(c3ccc[nH+]c3)C2)c1,[C12454],2
89478,c1cncc(C2CC[NH2+]C(c3cccnc3)C2)c1,[C12454],1
89479,c1nc(CC2CCNCC2)c[n-]1,[C17932],-1


In [129]:
df_cat = df[df['formal_charge'] > 0]
df_an = df[df['formal_charge'] < 0]

In [130]:
df_ilcom = pd.read_csv('data/commercial-il.tsv', delimiter='\t')

In [131]:
desc_names = [x[0] for x in Descriptors._descList]
def calc_descriptors(smiles):
    calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
    mol = Chem.MolFromSmiles(smiles)
    desc = calc.CalcDescriptors(mol)
    return desc

SELECTING CATIONS:

In [132]:
cat_desc = pd.DataFrame(df_ilcom.drop_duplicates(subset='cation')['cation'].apply(calc_descriptors).tolist(), columns=desc_names).add_suffix('_cation').fillna(0)
cat_desc = cat_desc.loc[:,cat_desc.nunique()>1]
scaler = StandardScaler()
cat_desc_norm = scaler.fit_transform(cat_desc)
cat_desc = pd.DataFrame(cat_desc_norm, columns=cat_desc.columns)

In [133]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(cat_desc)

pca_evr = pca.explained_variance_ratio_
cumsum = np.cumsum(pca_evr)

# Get dimensions where var >= 95% and values for variance at 2D, 3D
dim_95 = np.argmax(cumsum >= 0.95) + 1

cat_commercial_pca = pca.transform(cat_desc)[:,:dim_95]

In [134]:
nat_cat_desc = pd.DataFrame(df_cat['smiles'].apply(calc_descriptors).tolist(), columns=desc_names).add_suffix('_cation').fillna(0)
nat_cat_desc = nat_cat_desc[nat_cat_desc.columns.intersection(cat_desc.columns)]

[15:36:04] WARNING: not removing hydrogen atom without neighbors
[15:36:04] WARNING: not removing hydrogen atom without neighbors
Traceback (most recent call last):
  File "/home/lunet/cgvk2/miniconda3/envs/ionic-liquids/lib/python3.13/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
             ~~^^^^^
  File "/home/lunet/cgvk2/miniconda3/envs/ionic-liquids/lib/python3.13/site-packages/rdkit/Chem/SpacialScore.py", line 72, in SPS
    return _SpacialScore(mol, normalize=normalize).score
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lunet/cgvk2/miniconda3/envs/ionic-liquids/lib/python3.13/site-packages/rdkit/Chem/SpacialScore.py", line 95, in __init__
    self.score /= self.mol.GetNumHeavyAtoms()
ZeroDivisionError: division by zero


In [135]:
nat_cat_desc_norm = scaler.transform(nat_cat_desc)
nat_cat_desc = pd.DataFrame(nat_cat_desc_norm, columns=nat_cat_desc.columns)
cat_natural_pca = pca.transform(nat_cat_desc)[:,:dim_95]

In [136]:
min_distances = []
closest_points = []
for i, features in enumerate(cat_natural_pca):
    distances = np.linalg.norm(cat_commercial_pca - features, axis=1)
    min_distances.append(distances.min())
    closest_points.append(np.argmin(distances))
min_distances = np.array(min_distances)
closest_points = np.array(closest_points)

np.percentile(min_distances, 5)

np.float64(13.977418236080306)

In [137]:
#Print n most similar
n= 500
for nat, com, dist in zip(df_cat.iloc[min_distances.argsort()[:n]]['smiles'], df_ilcom.drop_duplicates(subset='cation').iloc[closest_points[min_distances.argsort()[:n]]]['cation'], min_distances[min_distances.argsort()[:n]]):
    print(com, nat, dist)

[K+] [K+] 0.0
[Mg+2] [Mg+2] 0.0
[Co+2] [Co+2] 0.0
C[N+](C)(C)CC(=O)O C[N+](C)(C)CC(=O)O 4.163336342344337e-17
C[N+](C)(C)CCO C[N+](C)(C)CCO 1.1102230246251565e-16
[Zn+2] [Zn+2] 2.0816681711721685e-16
[Na+] [Na+] 4.440892098500626e-16
CC[N+](CC)(CC)CC CC[N+](CC)(CC)CC 2.6645352591003757e-15
[Zn+2] [Fe+2] 0.4304401995710957
C[N+](C)(C)CC(=O)O C[N+](C)(C)C/C=C/C(=O)O 2.4209583610437595
[Mg+2] [Ca+2] 2.443031756078016
C[N+](C)(C)CC(=O)O C[N+](C)(C)CCCC(=O)O 3.2205297514145164
C=C(C)C(=O)OCC[N+](C)(C)CCCC CC(=O)OCC[N+](C)(C)C 4.532747457327806
CCC[NH+](C)CC C[NH2+]C 4.878625213303485
C[N+](C)(C)CC(=O)O C[NH+](C)CC(=O)O 4.896815728062814
CCCCCCCCCCCCCCCC[N+](C)(C)C C[N+](C)(C)CCCCCCCCCC[N+](C)(C)C 5.125660976131365
C[N+](C)(C)CC(=O)O C[NH2+]CC(=O)O 5.3065923574005085
CCC[NH+](C)CC [NH3+]C1CCCCC1 5.566228294189948
C[N+](C)(C)CC(=O)O C[NH2+]CCCC(=O)O 5.874717553093243
CCC[N+]1(C)CCCCC1 CCC[C@H]1CCCC[NH2+]1 5.897548726421771
C[N+](C)(C)CC(=O)O C[S+](C)CCC(=O)O 6.064165905505155
CCCC[N+](C)(C)C 

In [138]:
df_cat.iloc[min_distances.argsort()[:500]].to_csv('data/cations.tsv', sep='\t', index=False)

SELECTING ANIONS:

In [145]:
an_desc = pd.DataFrame(df_ilcom.drop_duplicates(subset='anion')['anion'].apply(calc_descriptors).tolist(), columns=desc_names).add_suffix('_anion').fillna(0)
an_desc = an_desc.loc[:,an_desc.nunique()>1]
scaler = StandardScaler()
an_desc_norm = scaler.fit_transform(an_desc)
an_desc = pd.DataFrame(an_desc_norm, columns=an_desc.columns)

In [146]:
pca = PCA(n_components=30)
pca.fit(an_desc)

pca_evr = pca.explained_variance_ratio_
cumsum = np.cumsum(pca_evr)

# Get dimensions where var >= 95% and values for variance at 2D, 3D
dim_95 = np.argmax(cumsum >= 0.95) + 1

an_commercial_pca = pca.transform(an_desc)[:,:dim_95]

In [147]:
nat_an_desc = pd.DataFrame(df_an['smiles'].apply(calc_descriptors).tolist(), columns=desc_names).add_suffix('_anion').fillna(0)
nat_an_desc = nat_an_desc[nat_an_desc.columns.intersection(an_desc.columns)]

In [148]:
nat_an_desc_norm = scaler.transform(nat_an_desc)
nat_an_desc = pd.DataFrame(nat_an_desc_norm, columns=nat_an_desc.columns)
an_natural_pca = pca.transform(nat_an_desc)[:,:dim_95]

In [149]:
min_distances = []
closest_points = []
for i, features in enumerate(an_natural_pca):
    distances = np.linalg.norm(an_commercial_pca - features, axis=1)
    min_distances.append(distances.min())
    closest_points.append(np.argmin(distances))
min_distances = np.array(min_distances)
closest_points = np.array(closest_points)

np.percentile(min_distances, 5)

np.float64(7.16431231603666)

In [150]:
#Print n most similar
n= 1000
for nat, com, dist in zip(df_an.iloc[min_distances.argsort()[:n]]['smiles'], df_ilcom.drop_duplicates(subset='anion').iloc[closest_points[min_distances.argsort()[:n]]]['anion'], min_distances[min_distances.argsort()[:n]]):
    print(com, nat, dist)

O=P([O-])(O)O O=P([O-])(O)O 0.0
[I-] [I-] 0.0
CCCCCCCCCC(=O)[O-] CCCCCCCCCC(=O)[O-] 0.0
CCOP(=O)([O-])OCC CCOP(=O)([O-])OCC 0.0
[Cl-] [Cl-] 0.0
CC(=O)[O-] CC(=O)[O-] 0.0
NCCCC[C@H](N)C(=O)[O-] NCCCC[C@H](N)C(=O)[O-] 0.0
CS(=O)(=O)[O-] CS(=O)(=O)[O-] 0.0
O=[N+]([O-])[O-] O=[N+]([O-])[O-] 0.0
NCCCC[C@H](N)C(=O)[O-] NCCCC[C@@H](N)C(=O)[O-] 0.0
CCCCCCCC(=O)[O-] CCCCCCCC(=O)[O-] 0.0
CCC(=O)[O-] CCC(=O)[O-] 0.0
Cc1ccc(S(=O)(=O)[O-])cc1 Cc1ccc(S(=O)(=O)[O-])cc1 0.0
NCCCC[C@H](N)C(=O)[O-] N[C@@H](CCCCNO)C(=O)[O-] 1.5181282330341628
CCC(=O)[O-] CCCC(=O)[O-] 1.5315005432373117
NCCCC[C@H](N)C(=O)[O-] NCCC[C@@H](N)C(=O)[O-] 1.6223661547638135
NCCCC[C@H](N)C(=O)[O-] NCCC[C@H](N)C(=O)[O-] 1.6223661547638135
NCCCC[C@H](N)C(=O)[O-] C[C@H](N)CC[C@@H](N)C(=O)[O-] 1.9140326239323564
CCC(=O)[O-] CC(C)C(=O)[O-] 2.06145143190591
CC(=O)[O-] C=CC(=O)[O-] 2.316546685126556
CP(=O)([O-])[O-] O=P([O-])([O-])CO 2.4452245145246647
O=[N+]([O-])[O-] O=N[O-] 2.452422381932278
CCC(=O)[O-] O=C([O-])C1CC1 2.5140732464416

In [151]:
df_an.iloc[min_distances.argsort()[:1000]].to_csv('data/anions.tsv', sep='\t', index=False)